# Kapitel 7 - Hyperparameteroptimierung

## 7.1. Kapitelübersicht <a class="anchor" id="7-1"/>

TODO
https://de.wikipedia.org/wiki/Hyperparameteroptimierung

<b>Abschnittsübersicht</b><br>

[7.1. Kapitelübersicht](#7-1)<br>

TODO<br>
Bis jetzt hatten wir nur die standard parameter von scikit learn bei unseren klassifizierungsverfahren genutzt. selbst hatten wir keine parameter übergeben. durch gezieltes parameter tuning können klassifizierungsverfahren jedoch noch einmal verbessert werden. in diesem abschnitt werden wir uns zunächst erneut die verfahren naive bayes und log reg angucken und deren parameter in der dokumentation. intuitionen hatten wir schon in den einzelnen Kapiteln besprochen.

## 7.2. Hyperparameter von Multinomial Naive Bayes

Implementieren wir zunächst ein weiteres Mal das <b>Multinomial Naive Bayes</b> Klassifizierungsverfahren. Hier haben wir es in einer Funktion zusammengefasst, um den Code kleiner zu halten (AF?).

In [1]:
import pandas as pd
corpus = pd.read_csv("tutorialdata/corpora/wikicorpus_v2.csv", index_col=0)

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
import numpy as np


labels = LabelEncoder().fit_transform(corpus["category"])
vector  = TfidfVectorizer().fit_transform(corpus["text"])


X_train, X_test, y_train, y_test = train_test_split(vector, 
                                                    labels, 
                                                    test_size=0.2, 
                                                    train_size=0.8,
                                                    random_state=42)

def classify_mnb(alpha=1.0):
    
    # Multinomial Naive Bayes
    mnb_classifier = MultinomialNB(alpha)
    mnb = mnb_classifier.fit(X_train, y_train)

    # cross validation des Trainingsdatensatzes
    mnb_scores = cross_val_score(mnb_classifier, vector, labels, cv=3)
    mnb_mean = np.mean(mnb_scores)

    print("Der Mittelwert der cross validation bei der  Klassifizierung " 
          + f" mit Multinomial Naive Bayes ist {str(np.around(mnb_mean, decimals=3))}."
          + "\n")


    # F1-score des Testdatensatzes
    y_pred = mnb_classifier.predict(X_test)
    mnb_f1 = f1_score(y_test, y_pred, average="micro")

    print("Der F1-score für die Klassifizierung mit Multinomial Naive Bayes ist "
          + f"{str(np.around(mnb_f1, decimals=3))}.")
classify_mnb()

Der Mittelwert der cross validation bei der  Klassifizierung  mit Multinomial Naive Bayes ist 0.859.

Der F1-score für die Klassifizierung mit Multinomial Naive Bayes ist 0.87.


Schauen wir uns zunächst die möglichen Parameter von Multinomial Naive Bayes in der <a href="https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html">Dokumentation</a> an. Es gibt drei Parameter:
- `alpha` (default = 1.0)
- `fit_prior` (default = True)
- `class_prior` (default = None)<br>

Wir werden uns nur `alpha` anschauen. Ist `alpha = 1`, wir <b>Laplace Smoothing</b> angewandt (siehe Kapitel 3), d.h. jede Worthäufigkeit wird um 1 erhöht. Ist `alpha < 1`, wird <b>Lidstone Smoothing</b> angewandt, das im Grunde das Gleiche ist. Ist `alpha = 0`, wird gar kein Smoothing angewandt. Standardmäßig ist `alpha = 1` ausgewählt, weshalb wir hier zunächst `alpha = 0.5` setzen.

In [9]:
classify_mnb(0.5)

Der Mittelwert der cross validation bei der  Klassifizierung  mit Multinomial Naive Bayes ist 0.876.

Der F1-score für die Klassifizierung mit Multinomial Naive Bayes ist 0.889.


Sowohl der Mittelwert der cross validation als auch der F1-score haben sich verbessert. Versuchen wir es nun mit `alpha = 0.1`.

In [10]:
classify_mnb(0.1)

Der Mittelwert der cross validation bei der  Klassifizierung  mit Multinomial Naive Bayes ist 0.898.

Der F1-score für die Klassifizierung mit Multinomial Naive Bayes ist 0.906.


Erneut können wir eine Verbesserung feststellen. Um weitere Parameter zu testen, können wir nun zwei Strategien verfolgen: Einfach weiter Parameter ausprobieren oder ein Hyperparameteroptimierungsverfahren benutzen, um die optimalen Hyperparameter zu finden (AF).

## 7.3. Grid Search

<b>Grid Search</b> (deutsch: Rastersuche) ist ein Suchverfahren, dass mithilfe der <b>Brute-Force-Methode</b> (auch <i>erschöpfende Suche</i> genannt) die optimalen Hyperparameter sucht. Dabei muss ein selbst erstelltes Dictionary von möglichen Hyperparametern als keys übergeben werden (hier `parameters`), deren values mögliche Parameterwerte sind. Grid Search nutzt als Evaluationsmethode die <b>cross validation</b>, weshalb man die Anzahl der Teildatensätze (<i>folds</i>) beim Parameter `cv` angeben muss. Zudem müssen wir, um den <b>F1-score nutzen</b> zu können, diesen beim Parameter `scoring` angeben. Grid Search liefert gute Werte, leidet aber unter dem <b>Fluch der Dimensionalität</b>.

<div class="alert alert-info">
<b>Exkurs:</b> Fluch der Dimensionalität
    
Der <b>Fluch der Dimensionalität</b> (englisch: curse of dimensionality) ist ein Begriff, der auf die Schwierigkeiten beim Anpassen von Modellen, bei der Schätzung von Parametern oder bei der Optimierung einer Funktion in vielen Dimensionen verweisen soll. Je mehr Dimensionen ein Eingabedatenraum hat, umso schwieriger wird es, optimale Parameter für diesen Raum zu finden. Bei den Klassifizierungsverfahren bedeutet dies konkret: Je mehr Hyperparameter ein Klassifizierungsverfahren hat, umso schwieriger wird es, diese zu optimieren, um bspw. einen idealen F1-score zu erreichen.

In [26]:
from sklearn.model_selection import GridSearchCV

parameters = {"alpha": np.array([0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 
                0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])}


grid = GridSearchCV(mnb_classifier, parameters, cv=5, scoring="f1_micro")
grid.fit(X_train, y_train)

# Ergebnisse
print(f"Der beste Hyperparameter für alpha ist {str(grid.best_estimator_.alpha)}.")
print(f"Der beste Score ist {str(np.around(grid.best_score_, decimals=4))}.")

Der beste Hyperparameter für alpha ist 0.01.
Der beste Score ist 0.9302.


Von unseren Hyperparametern scheint `0.01` der beste Hyperparameter zu sein. Wir können nun die umliegenden Werte zu `0.01` als neue Parameter auswählen und GridSearch übergeben.

In [24]:
parameters = {"alpha": np.array([0.001, 0.002, 0.003, 0.004, 0.005, 0.006,
                                 0.007, 0.008, 0.009, 0.01, 0.011, 0.012,
                                0.013, 0.014, 0.015, 0.016, 0.017, 0.018, 
                                0.019, 0.2])}

grid = GridSearchCV(mnb_classifier, parameters, cv=5, scoring="f1_micro")
grid.fit(X_train, y_train)

# Ergebnisse
print(f"Der beste Hyperparameter für alpha ist {str(grid.best_estimator_.alpha)}.")
print(f"Der beste Score ist {str(np.around(grid.best_score_, decimals=4))}.")

Der beste Hyperparameter für alpha ist 0.006.
Der beste Score ist 0.9317.


Ein noch besserer Wert als `0.01` scheint `0.006` zu sein, der Score hat sich von `0.9302` auf `0.9317` erhöht. Es stellt sich nun die Frage, ob sich der Aufwand für eine so geringe Verbesserung lohnt. Eine eindeutige Antwort gibt es darauf nicht, eine Verbesserung anzustreben, die kleiner als `0.01` ist, ist für unsere Zwecke nicht sonderlich sinnvoll. (ICH: hier was sagen zu überoptimierung zitat)

## 7.4. Random Search

https://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

https://machinelearningmastery.com/difference-between-a-parameter-and-a-hyperparameter/

Schauen wir uns zunächst die möglichen Parameter von Multinomial Naive Bayes in der <a href="https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html">Dokumentation</a> an. 

In [ ]:
from sklearn.model_selection import GridSearchCV

alphas = np.array([0.000001, 0.00001, 0.0001, 0.001, 0.01, 
                0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
parameters = {'alpha': alphas, 
              'fit_prior' : [True, False], 
              'class_prior ' : [None, [.1,.9],[.2, .8]]}


grid = GridSearchCV(estimator=mnb_classifier, param_grid=dict(alpha=alphas), cv=5)
grid.fit(X_train, y_train)
print(grid)
# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_.alpha)

## 7.? Mögliche Fehler

<b>GridSearch</b>:
- F1-score soll genutzt werden, `scoring` Parameter wurde nicht definiert → `scoring`-Parameter angeben mit Wert "f1", "f1_micro", "f1_macro" etc.
- Fehlercode: `Target is multiclass but average='binary'. Please choose another average setting.` → Nur "f1" wurde beim Parameter `scoring` angegeben, obwohl die es mehr als zwei Klassen bei den Daten gibt. Standardmäßig geht "f1" von einer binären Klassifikation aus, deshalb muss z.B. "f1_micro" oder "f1_macro" angegeben werden.